In [ ]:
import os
import yaml
import json
import requests
import logging
from kafka import KafkaConsumer
from time import sleep


cfg_path = os.path.join("config_m3.yaml")
with open(cfg_path) as f:
    cfg = yaml.safe_load(f)


KAFKA_BOOT = cfg['kafka']['bootstrap_servers']
ALERT_TOPIC = cfg['kafka']['alert_topic']


ENABLE_TELEGRAM = cfg['notifier'].get('enable_telegram', False)
BOT_TOKEN = cfg['notifier']['telegram_bot_token']
CHAT_ID = cfg['notifier']['telegram_chat_id']


log_path = cfg['paths']['alerts_log']
log_dir = os.path.dirname(log_path)
os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


consumer = KafkaConsumer(
    ALERT_TOPIC,
    bootstrap_servers=KAFKA_BOOT,
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)


def send_telegram(message, retries=3):
    if not ENABLE_TELEGRAM or not BOT_TOKEN or not CHAT_ID:
        logging.warning("Telegram disabled or token/chat_id missing. Message not sent.")
        print("⚠️ Telegram disabled or token/chat_id missing.")
        print("Message:", message)
        return

    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    for attempt in range(1, retries + 1):
        try:
            response = requests.post(url, data={"chat_id": CHAT_ID, "text": message})
            if response.status_code == 200:
                logging.info(f"Telegram message sent: {message}")
                return
            else:
                logging.error(f"Telegram failed ({response.status_code}): {response.text}")
        except Exception as e:
            logging.error(f"Exception sending Telegram message: {e}")
        
        logging.info(f"Retry {attempt}/{retries} in 2s...")
        sleep(2)
    
    print("❌ Failed to send Telegram message after retries:", message)


for msg in consumer:
    alert = msg.value
    street = alert.get('street_name', '?')
    level = alert.get('alert_level', 'LOW')
    vehicles = alert.get('vehicles_per_window', 0)
    text = f"🚨 ALERT: {street} | Level={level} | Vehicles={vehicles}"

    logging.info(f"Received alert: {alert}")
    print("Notifier:", text)

    send_telegram(text)
